## Tarea 2 (10 puntos). Operaciones con RDDs
- Obtener información de algún origen de datos (propio o de API)
- Convertir el origen de datos a RDD con pySpark
- Realizar alguna operación en el RDD, como estadísticas descriptivas básicas
- Subir la práctica a un repositorio público y etiquetarla claramente

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

from pyspark.sql import SparkSession

In [2]:
import multiprocessing

multiprocessing.cpu_count()

16

In [3]:
from pyspark import SparkConf

In [4]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [25]:
conf = SparkConf().setAppName("Tarea2")\
    .setMaster("local") #\
    #.set("spark.executor.memory", "8g") #\
    #.set("spark.executor.cores","2")\
    

In [6]:
#sc = SparkContext("local", "RDDs")
#sc = SparkContext(conf= conf)

In [7]:
#spark = SparkSession(sparkContext= sc)

In [26]:
spark : SparkSession = SparkSession.builder.config(conf=conf).getOrCreate()

In [27]:
spark

In [29]:
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [30]:
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\data.zip')

path_guardar = pathlib.Path.joinpath(direccion.parent.parent, 'Datos')

archivos_datos = [
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2022.csv').as_posix(),
    pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2021.csv').as_posix()#,
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2020.csv').as_posix(),
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2019.csv').as_posix(),
    #pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2018.csv').as_posix()
]

archivos_datos_en_zip  = [
    #'Combined_Flights_2022.csv',
    'Combined_Flights_2021.csv'#,
    #'Combined_Flights_2020.csv',
    #'Combined_Flights_2019.csv',
    #'Combined_Flights_2018.csv'
]


In [12]:
#with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
#    for archivo in archivos_datos_en_zip:
#        archivo_zip.extract(archivo, path_guardar)

In [13]:
#def leer_archivo(archivo):
#    direccion = pathlib.Path.cwd()
#    direccion_archivo =  pathlib.Path.joinpath(direccion.parent.parent, 'Datos',archivo)
#    
#    return spark.read.csv(direccion_archivo)

In [14]:
#rdd = spark.sparkContext.parallelize(archivos_datos)
#rdd_contenido = rdd.map(lambda archivo: {"archivo": archivo,"datos": leer_archivo(archivo)})


In [15]:
#rdd = spark.sparkContext.parallelize(archivos_datos)
#rdd_contenido = rdd.map(lambda archivo: leer_archivo(archivo))

In [31]:
from pyspark import StorageLevel

In [32]:
rdd = spark.read.option('header','true').csv(archivos_datos).select(columnas).rdd

#rdd.saveAsTextFile("prueba_datos.txt")

In [18]:
#rdd_memoria = rdd.persist(StorageLevel.MEMORY_AND_DISK)
#rdd_reparticionado = rdd.repartition(50)

In [19]:
#rdd_reparticionado.getNumPartitions()

In [20]:
#rdd_reparticionado.context

In [33]:
rdd_memoria = rdd.persist(StorageLevel.MEMORY_AND_DISK)


In [34]:
rdd_memoria.first()

Row(Airline='SkyWest Airlines Inc.', CRSDepTime='724', DepTime='714.0', DepDelay='-10.0', CRSArrTime='843', ArrTime='818.0', ArrDelay='-25.0', ActualElapsedTime='64.0', CRSElapsedTime='79.0', Distance='262.0', Year='2021', Month='3', DayofMonth='3', DayOfWeek='3', Tail_Number='N728SK', Flight_Number_Operating_Airline='3133', Origin='SGU', OriginAirportID='14794', OriginAirportSeqID='1479405', OriginCityName='St. George, UT', OriginStateName='Utah', Dest='PHX', DestAirportID='14107', DestAirportSeqID='1410702', DestCityName='Phoenix, AZ', DestStateName='Arizona', Cancelled='False', Diverted='False')

# cantidad de filas:

In [35]:
rdd_memoria.count()

6311871

6,311,871 filas

In [36]:
rdd_por_aerolinea = rdd_memoria.groupBy(lambda x : x[0]).persist(StorageLevel.MEMORY_AND_DISK)


In [37]:
rdd_por_aerolinea.count()

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50106)
Traceback (most recent call last):
  File "C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "C:\Spark\spark-3.5.3-bin-hadoop3\python\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "C:\Spark\spark-3.5.3-bin-hadoop3\python\pyspark\accumulators.py", line 267, in poll
    if 

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "C:\Spark\spark-3.5.3-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\andre\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] Se ha forzado la interrupción de una conexión existente por el host remoto

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Spark\spark-3.5.3-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Spark\spark-3.5.3-bin-hadoop3\python\lib\py4j-0.10.9.7-src.zip\py4j\clien

ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

In [24]:
spark.stop()

In [22]:
#rdd_archivo = spark.sparkContext.textFile(archivos_datos[0]).persist(StorageLevel.MEMORY_AND_DISK)
#
#
#rdd_1 = rdd_archivo.coalesce(20)
#
#rdd_2 = rdd_1.map(lambda x : x.split(','))
#
#print(rdd_2)
#print(rdd_2.getNumPartitions())

In [23]:
#rdd_2.first()